<a href="https://colab.research.google.com/github/uguazelli/langchain/blob/main/sequential_chaining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequential Chaining (Multi-Step Prompting)
Use multi-step chaining if:
* You need very reliable results
* You want to inject logic or validation between steps
* You're building an agent or a dynamic flow

# Pip Install

In [1]:
# Install required packages
!pip install -U langchain langchain-community langchain-openai
!pip install pydantic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchain-core-0.3.65:
      Successfully uninstalled langchain-core-0.3.65
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25


# Imports

In [2]:
# Imports'
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI # ✅ use langchain_openai, not community
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
import json

In [3]:
# Get API key from Colab secrets
api_key = userdata.get('OPENAI_API_KEY')

In [4]:
# Define LLM (completion model like GPT-3.5-turbo-instruct and level of creativity 0.6 of 1)
llm = OpenAI(model="gpt-3.5-turbo-instruct", api_key=api_key, temperature=0.6)

In [5]:
# Test simple call
response = llm.invoke("Tell me a fun fact about AI")
print("Simple Response:", response)

Simple Response: 

One fun fact about AI is that it can learn and improve itself without human intervention. This is known as "machine learning" and it allows AI to continuously adapt and become more efficient at completing tasks. Another fun fact is that AI has been used to create original artworks and even a novel, showing its creative potential. Additionally, AI is being used in video games to create more realistic and challenging opponents for players.


In [6]:
# Step 1: Name
name_prompt = PromptTemplate.from_template(
    "I want to open a {topic} restaurant. Suggest a really nice name. Return just one name."
)

# Step 2: Slogan
slogan_prompt = PromptTemplate.from_template(
    "Create a catchy slogan for a restaurant named '{name}'."
)

# Step 3: Concept
concept_prompt = PromptTemplate.from_template(
    "Describe the branding and interior design concept for a restaurant named '{name}' with the slogan '{slogan}'."
)

# Step 4: Logo Description
logo_prompt = PromptTemplate.from_template(
    "Based on the name '{name}' and slogan '{slogan}', describe an ideal logo: colors, icon, font, and style."
)

# Step 5: Image Generation Prompt (for DALL·E or Midjourney)
image_prompt = PromptTemplate.from_template(
    "Create a visual prompt for an image generation tool to design the logo: '{logo_description}'"
)

In [7]:
# Build each step
step1 = name_prompt | llm
step2 = lambda name: slogan_prompt | llm
step3 = lambda name, slogan: concept_prompt | llm
step4 = lambda name, slogan: logo_prompt | llm
step5 = lambda logo_description: image_prompt | llm

In [8]:
# Wrap in RunnableSequence
def full_pipeline(topic: str):
    name = step1.invoke({"topic": topic}).strip()
    slogan = (slogan_prompt | llm).invoke({"name": name}).strip()
    concept = (concept_prompt | llm).invoke({"name": name, "slogan": slogan}).strip()
    logo = (logo_prompt | llm).invoke({"name": name, "slogan": slogan}).strip()
    image_prompt_text = (image_prompt | llm).invoke({"logo_description": logo}).strip()

    return {
        "name": name,
        "slogan": slogan,
        "concept": concept,
        "logo_description": logo,
        "image_prompt": image_prompt_text
    }

In [9]:
# Run
result = full_pipeline("Brazilian")
for key, value in result.items():
    print(f"\n🔹 {key.upper()}:\n{value}")


🔹 NAME:
Fogo de Chão

🔹 SLOGAN:
"Savor the fire, taste the passion at Fogo de Chão."

🔹 CONCEPT:
Branding Concept:
The branding concept for Fogo de Chão is centered around the idea of fire and passion. The name itself, which translates to "fire of the ground" in Portuguese, evokes images of a traditional Brazilian churrascaria, where meats are cooked over an open flame. The logo for Fogo de Chão features a stylized flame, with bold and vibrant colors, representing the passion and energy of Brazilian culture. The slogan, "Savor the fire, taste the passion at Fogo de Chão," further reinforces this concept, inviting customers to experience the fiery and passionate flavors of Brazil at the restaurant.

Interior Design Concept:
The interior design of Fogo de Chão is inspired by the rustic and warm atmosphere of traditional Brazilian churrascarias. The main dining area features a mix of warm earth tones and natural materials, such as wood and stone, creating a cozy and inviting ambiance. Th